# Calculate Clusters


## Set up


In [37]:
from pathlib import Path
import json

import altair as alt
import gbm
import vine
from pdpilot import partial_dependence

from clustering import create_dataset

In [ ]:
alt.data_transformers.enable("vegafusion")

In [39]:
output_dir = Path("results")
output_dir.mkdir(parents=True, exist_ok=True)

In [40]:
# random seed
seed = 1

## Create dataset


In [41]:
num_instances = 1000

In [42]:
df = create_dataset(num_instances=num_instances, seed=seed)
df.to_csv(output_dir / "data.csv", index=False)

In [43]:
df_X = df.drop(columns=["y"])
features = list(df_X.columns)
X = df_X.to_numpy()
y = df["y"].to_numpy()

In [44]:
def scatter(df):
    df_vis = df.copy()
    df_vis["cluster"] = (df_vis["x3"] >= 0) & (df_vis["x4"] >= (1 / 3))
    return (
        alt.Chart(df_vis)
        .mark_point(filled=True)
        .encode(
            x="x2",
            y="y",
            color=alt.Color("cluster")
            .scale(range=["#1b9e77", "#7570b3"])
            .legend(title="x3 >= 0 and x4 >= 1/3", orient="top", symbolOpacity=1),
        )
    )

In [ ]:
x2_feature_scatter = scatter(df)
x2_feature_scatter

In [46]:
x2_feature_scatter.save((output_dir / "x2-feature-scatter.png").as_posix(), ppi=200)

## Modeling


In [ ]:
# This takes about 70 seconds to run on my M1 Macbook Pro.
cv_results, booster = gbm.nested_cross_validation_and_train(
    X,
    y,
    features=features,
    nominal_features=[],
    objective="regression",
    jobs=4,
    seed=seed,
)

In [ ]:
booster.save_model(output_dir / "model.txt")

In [ ]:
cv_results["scores"] = cv_results["scores"].to_json(orient="records")
(output_dir / "cv_results.json").write_text(
    json.dumps(cv_results, indent=4), encoding="UTF-8"
)

In [ ]:
booster.params

In [ ]:
print(f"CV RMSE: {cv_results['mean_score']}")

## Calculate ICE plots


In [52]:
# number of points in a PDP/ICE line
resolution = 20

### PDPilot

First we have PDPilot use its default parameters for the decision trees, which have a max depth of 3.

We use PDPilot to calculate the ICE plots and clusters twice. First, we use its default parameters for the decision trees, which have a max depth of 3. Then we have PDPilot use decision trees with a max depth of 1.


In [ ]:
for decision_tree_params in [{"max_depth": 3, "ccp_alpha": 0.01}, {"max_depth": 1}]:
    pdpilot_output_path = (
        output_dir / f"pdpilot_max_depth_{decision_tree_params['max_depth']}.json"
    )
    partial_dependence(
        predict=booster.predict,
        df=df_X,
        features=features,
        resolution=resolution,
        decision_tree_params=decision_tree_params,
        seed=seed,
        n_jobs=1,
        output_path=pdpilot_output_path.as_posix(),
    )

### VINE

We run VINE with `n_clusters=2`, which is what we think the ICE plot for x2 should have, and with `n_clusters=5`, which is the default value. We do this both with and without `prune_clusters` set, which does additional pruning of the clusters.


In [54]:
for num_clusters, prune_clusters in [(2, True), (5, True), (2, False), (5, False)]:
    vine_output_path = (
        output_dir / f"vine_n_clusters_{num_clusters}_prune_{prune_clusters}.json"
    ).as_posix()

    vine.calculate_and_export(
        data=df_X,
        y=y,
        predict_func=booster.predict,
        num_clusters=num_clusters,
        num_grid_points=resolution,
        ice_curves_to_export=num_instances,
        cluster_method="good",
        prune_clusters=prune_clusters,
        seed=seed,
        output_path=vine_output_path,
    )